##### Mauricio Alexis Tinajero
##### 597186


# Análisis Exploratorio de Datos con el dataset de diamantes

En este notebook aprenderás a realizar un análisis exploratorio de datos (EDA) utilizando una base de datos SQLite con información sobre diamantes. Exploraremos tipos de variables, escalas de medición, estadísticas descriptivas, visualizaciones y transformaciones útiles para preparar los datos para modelos de machine learning.

---

### Conexión a la base de datos

### ¿Qué es Git?

Git es una herramienta que permite gestionar versiones de archivos, especialmente útil en proyectos de programación. Con Git puedes:

- Guardar cambios progresivos en tu trabajo
- Colaborar con otras personas sin perder el control de versiones
- Descargar (clonar) proyectos públicos desde plataformas como GitHub

En este notebook usamos `git clone` para copiar un repositorio que contiene bases de datos educativas en formato SQLite.

### ¿Qué es SQL?

SQL (Structured Query Language) es un lenguaje utilizado para consultar y manipular bases de datos. Permite extraer, filtrar, combinar y ordenar información de forma estructurada.

En este notebook usamos SQL para obtener datos de una base de datos SQLite. Algunas cláusulas clave que utilizamos son:

- `SELECT`: indica qué columnas queremos ver
- `FROM`: especifica de qué tabla se obtienen los datos
- `JOIN`: combina información de varias tablas relacionadas
- `ON`: define la condición de unión entre tablas
- `LIMIT`: restringe la cantidad de filas que se muestran (opcional)

Ejemplo usado:
```sql
SELECT carat, price, cut
FROM Observation
JOIN Cut ON Observation.cut_id = Cut.cut_id

In [2]:
# Clonar el repositorio
!git clone https://github.com/davidjamesknight/SQLite_databases_for_learning_data_science.git
%cd SQLite_databases_for_learning_data_science

# Conectarse a la base de datos e importar librerías

import sqlite3
import pandas as pd

db = sqlite3.connect('diamonds.db')
cursor = db.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

print("Tablas encontradas:")
for tabla in tablas:
    print(tabla[0])


fatal: destination path 'SQLite_databases_for_learning_data_science' already exists and is not an empty directory.
/workspaces/exploratory-data-analysis-597186/SQLite_databases_for_learning_data_science


Tablas encontradas:
Observation
Cut
Color
Clarity


In [3]:
# TODO Recorrer cada tabla y mostrar sus columnas
for tabla in tablas:
    nombre_tabla = tabla[0]  # Extraer el nombre como cadena
    print(f"\nColumnas en la tabla '{nombre_tabla}':")

    # Usar PRAGMA para obtener información de las columnas
    cursor.execute(f"PRAGMA table_info('{nombre_tabla}');")
    columnas = cursor.fetchall()

    # Mostrar nombre y tipo de cada columna
    for columna in columnas:
        print(f" - {columna[1]} ({columna[2]})")  # columna[1] = nombre, columna[2] = tipo
        


Columnas en la tabla 'Observation':
 - carat (FLOAT)
 - depth (FLOAT)
 - table (FLOAT)
 - price (BIGINT)
 - x (FLOAT)
 - y (FLOAT)
 - z (FLOAT)
 - cut_id (BIGINT)
 - color_id (BIGINT)
 - clarity_id (BIGINT)

Columnas en la tabla 'Cut':
 - cut_id (BIGINT)
 - cut (TEXT)

Columnas en la tabla 'Color':
 - color_id (BIGINT)
 - color (TEXT)

Columnas en la tabla 'Clarity':
 - clarity_id (BIGINT)
 - clarity (TEXT)


# Generar primer SQL query.

In [32]:
# TODO: Crear la consulta SQL
sql = """
SELECT
    O.carat, 
    O.price,
    "O"."table",
    O.x,
    O.y,
    O.z,
    C.cut_id,
    Co.color_id,
    Cl.clarity_id

FROM
    Observation AS O
JOIN
    Cut AS C
    ON O.cut_id = C.cut_id
JOIN
    Color AS Co
    ON O.color_id = Co.color_id
JOIN
    Clarity AS Cl
    ON O.clarity_id = Cl.clarity_id
"""

# TODO: Almacenar datos en dataframe (df)
df = pd.read_sql_query(sql, db)
df.head()

,carat,price,table,x,y,z,cut_id,color_id,clarity_id
0,0.23,326,55.0,3.95,3.98,2.43,2,1,3
1,0.21,326,61.0,3.89,3.84,2.31,3,1,2
2,0.23,327,65.0,4.05,4.07,2.31,1,1,4
3,0.29,334,58.0,4.20,4.23,2.63,3,5,5
4,0.31,335,58.0,4.34,4.35,2.75,1,6,3


### Identificar tipos de datos

In [5]:
# TODO: Identifica tipos de dato
df.dtypes


carat         float64
price           int64
table         float64
x             float64
y             float64
z             float64
cut_id          int64
color_id        int64
clarity_id      int64
dtype: object

In [6]:
# TODO: Separa por numéricas y categóricas
numericas = df.select_dtypes(
    include = ['float64', 'int64']
).columns.tolist()

categoricas = df.select_dtypes(
    include = ['object']
).columns.tolist()

print("variables numericas", numericas)
print("variables categoricas", categoricas)

variables numericas ['carat', 'price', 'table', 'x', 'y', 'z', 'cut_id', 'color_id', 'clarity_id']
variables categoricas []


### Escala de medición de las variables

| Variable         | Tipo de dato | Escala de medición | Justificación |
|------------------|--------------|---------------------|----------------|
| `carat`          | Numérica     | Razón               | Tiene cero absoluto, se puede multiplicar/dividir |
| `price`          | Numérica     | Razón               | Representa valor monetario, cero tiene significado |
| `x`, `y`, `z`     | Numérica     | Razón               | Medidas físicas, cero indica ausencia |
| `depth`          | Numérica     | Intervalo           | Porcentaje relativo, no tiene cero absoluto claro |
| `"table"`        | Numérica     | Intervalo           | Porcentaje relativo, no tiene cero absoluto claro |
| `cut`            | Categórica   | Ordinal             | Tiene orden lógico: Fair < Good < Very Good < Ideal < Premium |
| `color`          | Categórica   | Ordinal o Nominal           | Escala gemológica: D (mejor) a J (peor) |
| `clarity`        | Categórica   | Ordinal o Nominal           | Escala gemológica: FL > IF > VVS1 > ... > I3 |


### Estadísticas Descriptivas

In [23]:
# TODO: Obten estadísticas desceiptivas por medio del metodo de pandas
df.describe().T

,count,mean,std,min,25%,50%,75%,max
carat,53940.0,0.797940,0.474011,0.2,0.40,0.70,1.04,5.01
price,53940.0,3932.799722,3989.439738,326.0,950.00,2401.00,5324.25,18823.00
table,53940.0,57.457184,2.234491,43.0,56.00,57.00,59.00,95.00
x,53940.0,5.731157,1.121761,0.0,4.71,5.70,6.54,10.74
y,53940.0,5.734526,1.142135,0.0,4.72,5.71,6.54,58.90
z,53940.0,3.538734,0.705699,0.0,2.91,3.53,4.04,31.80
cut_id,53940.0,2.553003,1.027708,0.0,2.00,2.00,3.00,4.00
color_id,53940.0,2.594197,1.701105,0.0,1.00,3.00,4.00,6.00
clarity_id,53940.0,3.835150,1.724591,0.0,2.00,4.00,5.00,7.00


In [28]:
# TODO: Obten conteo de valores para cada categoría de las variables catgeóricas
for col in numericas:
    print(df[col].value_counts())

carat
0.30    2604
0.31    2249
1.01    2242
0.70    1981
0.32    1840
        ... 
2.64       1
5.01       1
4.50       1
2.67       1
3.51       1
Name: count, Length: 273, dtype: int64
price
605     132
802     127
625     126
828     125
776     124
       ... 
2727      1
2736      1
2740      1
2747      1
2750      1
Name: count, Length: 11602, dtype: int64
table
56.0    9881
57.0    9724
58.0    8369
59.0    6572
55.0    6268
        ... 
63.4       1
62.1       1
79.0       1
76.0       1
61.6       1
Name: count, Length: 127, dtype: int64
x
4.37    448
4.34    437
4.33    429
4.38    428
4.32    425
       ... 
9.35      1
9.41      1
3.74      1
3.76      1
3.77      1
Name: count, Length: 554, dtype: int64
y
4.34     437
4.37     435
4.35     425
4.33     421
4.32     414
        ... 
3.87       1
3.83       1
3.81       1
3.82       1
31.80      1
Name: count, Length: 552, dtype: int64
z
2.70     767
2.69     748
2.71     738
2.68     730
2.72     697
        ... 
2.28    

- ¿Qué tipo de codificación sería más adecuada para cada variable categórica?
    - One-hot Encoding (dummies) -> Cut
    - Target Encoding (alta cardinalidad) -> Clarity
    - Label Encoding (ordinales) -> Color


### Correlaciones entre variables numéricoas

In [37]:
# TODO: Crear Matriz de Correlación con plotly (sólo la primera vez)
import plotly.figure_factory as ff

corr_matrix = df[numericas].corr().round(2)

fig = ff.create_annotated_heatmap(
    z = corr_matrix.values,
    x = corr_matrix.columns.tolist(),
    y = corr_matrix.index.tolist()
)
fig.update_layout(title="Matriz de Correlacao")
fig.show()

- ¿Qué variables parecen tener una relación fuerte con el precio?

### Visualizaciones con plotly

In [38]:
# TODO: Crear Scatter Plot
import plotly.express as px

fig = px.scatter(
    data_frame = df,
    x = 'carat',
    y = 'price',
    color = 'cut',
    hover_data = ['color', 'clarity'] 
)
fig.show()

ValueError: Value of 'hover_data_0' is not the name of a column in 'data_frame'. Expected one of ['carat', 'price', 'table', 'x', 'y', 'z', 'cut_id', 'color_id', 'clarity_id'] but received: color

### Boxplot pot tipo de corte

In [11]:
# TODO: Crear boxplot por tipo de corte

- ¿Qué tipo de corte tiene mayor dispersión en precios?

### Histogramas de variables numericas

In [12]:
# TODO: Histogramas de variables numéricas

- ¿Qué variables numéricas muestran distribución sesgada?


### Mapa de calor con correlaciones

### Detección de outliers utilizando RIC (IQR)

In [13]:
# TODO: Obtener numero de outliers por variable numérica

### Análisis de Distribución

In [14]:
# TODO: Obtener sesgo y curtosis de variables numéricas

### Transformación de variables categóricas

In [15]:
# TODO: Label Encoding con sklearn

El órden lógico de esta columna es Fair < Good < Very Good < Ideal < Premium

In [16]:
# TODO: Label Encoding Manual

In [17]:
# TODO: One-Hot Encoding

In [18]:
# TODO: Target Encoder
import category_encoders as ce

### Escalamiento de Datos

### ¿Qué es el escalamiento de datos?

El escalamiento de datos es una técnica que ajusta los valores numéricos para que estén en rangos comparables. Esto es útil cuando usamos algoritmos que son sensibles a las magnitudes de los datos (como regresiones o clustering).

A continuación, se explican tres métodos comunes:

---

#### 🔹 MinMaxScaler

- Ajusta los valores para que estén entre un mínimo y un máximo (por defecto, entre 0 y 1).
- Fórmula:  
  $$
  X_{\text{escalado}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}
  $$
- Útil cuando queremos conservar la forma original de la distribución.

---

#### 🔹 StandardScaler

- Centra los datos en 0 y los escala para que tengan una desviación estándar de 1.
- Fórmula:  
  $$
  X_{\text{escalado}} = \frac{X - \mu}{\sigma}
  $$
  donde $\mu$ es la media y $\sigma$ la desviación estándar.
- Ideal cuando los datos tienen una distribución aproximadamente normal.

---

En resumen: estos métodos ayudan a que los datos sean más comparables y útiles para análisis estadísticos o modelos de machine learning.

### ¿Se pueden combinar distintos escaladores?

Sí, es válido aplicar diferentes métodos de escalamiento a distintas columnas, especialmente cuando:

- Las variables tienen **naturaleza distinta** (por ejemplo, ingresos vs. proporciones).
- Algunas columnas tienen **valores extremos (outliers)** y otras no.
- Quieres conservar la **forma original** de ciertas distribuciones (MinMaxScaler) pero estandarizar otras (StandardScaler).

Ejemplo práctico:
- Usar `StandardScaler` en columnas como "ingresos" o "edad", que tienen distribución normal.
- Usar `MinMaxScaler` en columnas como "porcentaje de cumplimiento" o "calificaciones", que ya están en rangos definidos.

---

### Consideraciones importantes

- Si usas modelos **basados en distancia** (como KNN o clustering), asegúrate de que las escalas no generen sesgos. En ese caso, es mejor que todas las variables estén en rangos comparables.
- Para modelos como **árboles de decisión o random forest**, el escalamiento no es necesario, ya que no dependen de magnitudes.

---

En resumen: puedes escalar columnas de forma diferente, pero asegúrate de que tenga sentido para el tipo de análisis o modelo que estás usando.

# Pruebas de Hipótesis: Una y Dos Medias

Las **pruebas de hipótesis** nos ayudan a tomar decisiones sobre una población usando datos de una muestra. En particular, podemos comparar medias para saber si hay diferencias significativas o si los resultados podrían deberse al azar.

## ¿Qué es el estadístico t y el valor p?

- **Estadístico t:** Es una medida que compara la diferencia observada entre medias (o entre una media y un valor de referencia) con la variabilidad de los datos. Nos dice cuántas "desviaciones estándar" está la diferencia observada respecto a lo que esperaríamos por azar.
- **Valor p:** Es la probabilidad de obtener un resultado igual o más extremo que el observado, suponiendo que la hipótesis nula es cierta. Si el valor p es pequeño (por ejemplo, menor a 0.05), consideramos que la diferencia es significativa.

### Fórmulas

**Para una muestra:**

$$
t = \frac{\bar{x} - \mu_0}{s / \sqrt{n}}
$$

donde:

- $\bar{x}$ = media muestral  
- $\mu_0$ = media bajo la hipótesis nula  
- $s$ = desviación estándar muestral  
- $n$ = tamaño de la muestra  

**Para dos muestras independientes:**

$$
t = \frac{\bar{x}_1 - \bar{x}_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$

donde los subíndices 1 y 2 corresponden a cada grupo.

---

In [19]:
# TODO: Prueba de hipótesis para una media (t-test de una muestra)


## 2. Prueba de Hipótesis para dos Medias

Se utiliza para comparar si las medias de dos grupos son iguales.

**Ejemplo:**  
¿El precio promedio de los diamantes con corte 'Ideal' es igual al de los de corte 'Premium'?

- **Hipótesis nula ($H_0$):** $\mu_1 = \mu_2$
- **Hipótesis alternativa ($H_1$):** $\mu_1 \neq \mu_2$

In [20]:
# TODO: Prueba de hipótesis para dos medias independientes (t-test de dos muestras)